# Experiments driver for HuGaDB

This notebook collects the experiments performed for TIME 2025.

If you are running the code of each cell in the REPL, please make sure your working
directory is the same as this file by executing `; pwd`, or adjust the relative paths
accordingly to your system.

More about the dataset on https://github.com/romanchereshnev/HuGaDB.

In [1]:
import Pkg
using Serialization: serialize, deserialize

# activate the general environment of ModalAssociationRules.jl package
Pkg.activate(joinpath("..", "..", ".."))

# a collection of utilities
include(joinpath("..", "experiments-driver.jl"));

  Activating project at `~/.julia/dev/ModalAssociationRules`


In [ ]:
# algorithm to be used for mining
miningalgo = apriori

# distance function to compare a time series and a motif of the same length; 
# we suggest to use Z-Normalized Euclidean distance if you are not interested in the scale
# but just in identifying common shapes between signals.
expdistance = (x, y) -> zeuclidean(x, y) |> first;

# variables considered for these experiments
expvars = [ # accelerometer data
    1,3, # right foot x (+behind/-front) and z (+down/-up)  
    13,15, # right thigh x (+up/-down) and z (+behind/-front)
    19,21, # left foot x (+behind/-front) and z (+down/-up)
    31,33  # left thigh x (+up/-down) and z (+behind/-front)
]

# files selected to build the experiments DataFrame
expfiles = ["HuGaDB_v2_various_02_00.txt", "HuGaDB_v2_various_02_01.txt", 
    "HuGaDB_v2_various_02_02.txt", "HuGaDB_v2_various_02_03.txt", "HuGaDB_v2_various_02_04.txt", 
    "HuGaDB_v2_various_02_05.txt", "HuGaDB_v2_various_02_06.txt", "HuGaDB_v2_various_06_00.txt", 
    "HuGaDB_v2_various_06_01.txt", "HuGaDB_v2_various_06_02.txt", "HuGaDB_v2_various_06_03.txt", 
    "HuGaDB_v2_various_06_04.txt", "HuGaDB_v2_various_06_05.txt", "HuGaDB_v2_various_06_06.txt", 
    "HuGaDB_v2_various_06_07.txt", "HuGaDB_v2_various_06_08.txt", "HuGaDB_v2_various_06_09.txt", 
    "HuGaDB_v2_various_06_10.txt", "HuGaDB_v2_various_06_11.txt", "HuGaDB_v2_various_06_12.txt", 
    "HuGaDB_v2_various_06_13.txt", "HuGaDB_v2_various_06_14.txt", "HuGaDB_v2_various_06_15.txt", 
    "HuGaDB_v2_various_06_16.txt",  "HuGaDB_v2_various_06_17.txt", "HuGaDB_v2_various_06_18.txt", 
    "HuGaDB_v2_various_06_19.txt",  "HuGaDB_v2_various_06_20.txt", "HuGaDB_v2_various_06_21.txt", 
    "HuGaDB_v2_various_06_22.txt", "HuGaDB_v2_various_06_23.txt", "HuGaDB_v2_various_06_24.txt", 
    "HuGaDB_v2_various_06_25.txt","HuGaDB_v2_various_06_26.txt", "HuGaDB_v2_various_06_27.txt"
]

In [ ]:
# load all the data
X, (activity_strings, activity_ids), variablenames = load_hugadb(expfiles)

## Walking

In [ ]:
X_walking = filter_hugadb(X, 1) # 1 is the activity id for walking

In [ ]:
# run this cell if you need to create your literals, that is,
# to extract snippets from time series and give them a name.

ids, motifs, featurenames = label_motifs(
    X_walking, expvars, "HuGaDB-Walking";
    m1=25, n1=2,   # we want 2 snippets of length 100
    m2=50, n2=2    # and 2 snippets of length 300
)

In [6]:
# run this cell if you already created your literals
ids, motifs, featurenames = load_motifs("NATOPS-IHCC");

In [8]:
# run the experiment (hyperparameters are `initialize_experiment` kwargs) & save the results
logiset, miner = initialize_experiment(ids, motifs, featurenames, IHCC)
experiment!(miner, "i_have_command")

Mining...
Mining duration: 2912.19
Generating rules...
Generation duration: 634.22
Writing to: /home/mauro/.julia/dev/ModalAssociationRules/test/experiments/results/i_have_command


# Running

In [ ]:
X_walking = filter_hugadb(X, 2) # 2 is the activity id for running

In [ ]:
ids, motifs, featurenames = label_motifs(
    NCC, 
    vcat(collect(4:6), collect(10:12), 25),
    "NATOPS-NCC"
)

In [26]:
ids, motifs, featurenames = load_motifs("NATOPS-NCC");

In [10]:
logiset, miner = initialize_experiment(ids, motifs, featurenames, NCC)
experiment!(miner, "not_clear")

Mining...
Mining duration: 4826.88
Generating rules...
Generation duration: 1492.6
Writing to: /home/mauro/.julia/dev/ModalAssociationRules/test/experiments/results/not_clear


# Lock wings

In [ ]:
ids, motifs, featurenames = label_motifs(
    LWC, 
    vcat(collect(1:12)),
    "NATOPS-LWC"
)

In [25]:
ids, motifs, featurenames = load_motifs("NATOPS-LWC");

In [ ]:
logiset, miner = initialize_experiment(ids, motifs, featurenames, LWC)
experiment!(miner, "lock_wings")

In [ ]:
# fast sketch of the rules' output, sorted by lift 
# 
# ith = 1
# t = []
# 
# try
#     for r in arules(miner)[ith:length(arules(miner))]
#         _conf = round(
#             globalmemo(miner, (:gconfidence, r)), digits=2
#         )
#         _lift = round(
#             globalmemo(miner, (:glift, r)), digits=2
#         )
# 
#         push!(t, (r, _conf, _lift))
#         ith += 1
#     end
# catch
# end
# 
# f = open(joinpath("results", "lock_wings"), "w")
# for (r,c,l) in sort!(t, by=x->x[3], rev=true)
#     println(f, "$(r) $(c) $(l)");
# end
# flush(f)

# Plots

In [36]:
using Plots
using PGFPlotsX
pgfplotsx()

Plots.PGFPlotsXBackend()

In [23]:
IHCC_plot = plot(IHCC[1,4], IHCC[1,5], 
    label = "", linecolor = :blue, linewidth = 1, aspect_ratio = :equal,
    xlims=(0,3), ylims=(-2.2,1.5), 
    titlefont=font(18, "Arial"), guidefontsize=16, size = (300, 300),
    legend = :topright
)

plot!(IHCC[1,10], IHCC[1,11],
    label = "", linecolor = :red, linewidth = 1,)

xlabel!("x")
ylabel!("y")

title!("I have command")

savefig("results/plot_IHCC.tex");

In [ ]:

# actually this is not correct, as y axis should indicate a position...
# plot!(NCC[1,4], -NCC[1,25],
#     label = "Thumb orientation", linecolor = :red, linewidth = 1,)

# it is better to give the same idea but coloring the main signal in red, when the thumb 
# is down
x = NCC[1,4]; y = NCC[1,5];
x1 = x[1:15]; y1 = y[1:15];
x2 = x[15:33]; y2 = y[15:33];
x3 = x[33:end]; y3 = y[33:end];

NCC_plot = plot(x1, y1,
    label = "", linecolor = :blue, linewidth = 1,
    xlims=(0,3), ylims=(-2.2,1.5),
    titlefont=font(18, "Arial"), guidefontsize=16, size = (300, 300),
    # legend = :topleft
);


plot!(x2, y2, linecolor = :blue, linewidth = 1.5, label = "", ls=:dot)
plot!(x3, y3, linecolor = :blue, linewidth = 1, label = "")

plot!(NCC[1,10], NCC[1,11], linecolor = :red, linewidth = 1, label = "")

xlabel!("x")
ylabel!("y")

title!("Not clear")

savefig("results/plot_NCC.tex");

In [140]:
LWC_plot = plot(LWC[1,1], LWC[1,2],
    label = "", linecolor = :orange, linewidth = 1,
    xlims=(-1,3), ylims=(-2.2,1.5),
    titlefont=font(18, "Arial"), guidefontsize=16, size = (300, 300),
    legend = :topleft
);

plot!(LWC[1,4], LWC[1,5],
    label = "", linecolor = :blue, linewidth = 1,)

plot!(LWC[1,7], LWC[1,8],
    label = "", linecolor = :violet, linewidth = 1,)

plot!(LWC[1,10], LWC[1,11],
    label = "", linecolor = :red, linewidth = 1,)

xlabel!("x")
ylabel!("y")
zlabel!("z")

title!("Locked wings")

savefig("results/plot_LWC.tex");